### Dataset Loading and Preparation

In [4]:
from datasets import load_dataset, load_metric
from transformers import (
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)

### Reward Model

In [13]:
from src.reward.train_reward import load_trained_warp_reward_model

ModuleNotFoundError: No module named 'src'

Load trained model from Hugging Face. I've trained it earlier by using `create_train_reward_model` function in `src/reward/train_reward.py` file.

In [3]:
reward_tokenizer, reward_model = load_trained_warp_reward_model()

### SFT Model

#### Train and Eval dataset preparations

#### Model Training

In [27]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [28]:
sft_model_name = "lvwerra/gpt2-imdb"
sft_model = AutoModelForCausalLM.from_pretrained(sft_model_name)
sft_tokenizer = AutoTokenizer.from_pretrained(sft_model_name)

In [6]:
sft_tokenizer.pad_token = sft_tokenizer.eos_token

NameError: name 'sft_tokenizer' is not defined

### WARP Training

In [ ]:
from src.trainer.warp_config import WARPConfig
from src.trainer.warp_trainer import WARPTrainer

In [ ]:
warp_config = WARPConfig()

warp_trainer = WARPTrainer(
    config=warp_config,
    model=sft_model,
    tokenizer=sft_tokenizer,
    reward_model=reward_model,
    reward_tokenizer=reward_tokenizer
)

In [ ]:
warp_trainer.train()